# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (np.sum(l_c * l_c / l_s + r_c * r_c / r_s, axis=1)).reshape(-1, 1) / (l_s + r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        EPS = 1e-5
        return - (np.sum(l_c * np.log((l_c + EPS) / l_s) + 
                                r_c * np.log((r_c + EPS) / r_s), axis=1)).reshape(-1, 1) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (np.max(l_c, axis=1) + np.max(r_c, axis=1)).reshape(-1, 1) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(1 + n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __make_leaf(self, node_id, y):
        class_sizes = np.bincount(y, minlength=self.num_class)        
        self.tree[node_id] = [
            self.__class__.LEAF_TYPE, 
            np.argmax(class_sizes),
            class_sizes / np.sum(class_sizes).astype('float')
        ]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортрирует x и y по возрастанию x, определяет число различных классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Если убрать class_number и вместо него везде использовать self.num_class,
        # то будет работать не только для двух классов
        # Возможно, что с class_number тоже будет, но у меня так сделать не получилось
        # class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Обрезает отсортированный y так, чтобы при разделении нигде не осталось
        # меньше точек, чем задано в min_samples_split
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # Граничные индексы - где классы меняются (правая граница идущих подряд элементов одного класса...
        # ... которая после + (self.min_samples_split + 1) становится левой границей следующих элементов
        # для того, чтобы потом хорошо определились размеры частей при разбиении
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # Классы не меняются нигде
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # Находит число подряд идущих одинаковых эелементов (кроме последней группы, но это и не надо)
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        
        # r_border_ids.shape[0] - число мест, где можно разбить выборку
        # one_hot_code - матрица: i-я строка содержит 1 в j столбце, если i-й по счету отрезок имеет лейбл j
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        
        # Не просто one_hot, а еще и знаем, сколько элементов 
        # class_increments[i, j] = n, если i-й по счету отрезок содержит n элементов лейбла j
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # Добавили то, что отрезалось в начале sorted_y, когда делали splitted_sorted_y
        # чтобы знать, сколько элементов каждого класса остается слева при новом разбиении 
        # и не потерять то что сплитнули
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=self.num_class)
        
        # Что делает этот блок кода?
        # Считаем сколько элементов каждого класса будет оставаться слева и справа при разбиении по каждой границе
        l_class_count = np.cumsum(class_increments, axis=0) 
        # Было np.bincount(y), не работало для >2 классов
        r_class_count = np.bincount(y, minlength=self.num_class) - l_class_count
        
        # Сколько элементов всех классов будет оставаться слева  и справа при разбиении по каждой границе
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        
        # Что делает этот блок кода?
        # Ищем, при разбиении по какой границе impurity наилучшее
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # Номер элемента, с которого начнется правый класс при разбиении
        left_el_id = l_sizes[idx][0]
        # Вернем лучшее impurity и среднее между граничными элементами - threshold
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE +
        # self.NON_LEAF_TYPE

        # self.tree+
        # self.max_depth+
        # self.sufficient_share+
        # self.min_samples_split+

        # self.get_feature_ids+
        # self.__find_threshold+
        # self.__div_samples+
        # self.__fit_node+
        
        
        if depth == self.max_depth or y.shape[0] < 2 * self.min_samples_split:
            self.__make_leaf(node_id, y)
            return
        
        if np.max(np.bincount(y, minlength=self.num_class)) / float(y.shape[0]) > self.sufficient_share:
            self.__make_leaf(node_id, y)
            return        
        
        features = self.get_feature_ids(x.shape[1])
        thresholds = []
        impuritys = []
        
        for feature_id in features:
            gs, threshold = self.__find_threshold(x[:, feature_id], y)
            thresholds.append(threshold)
            impuritys.append(gs)
        
        best_feature = features[np.argmin(impuritys)]
        best_impurity = np.min(impuritys)
        best_threshold = thresholds[np.argmin(impuritys)]
        
        if best_impurity == float('+inf'):
            self.__make_leaf(node_id, y)
            return
            
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_feature, best_threshold)
                
        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            self.__make_leaf(node_id, y)
            return
        
        self.tree[node_id] = [
            self.__class__.NON_LEAF_TYPE,
            best_feature,
            best_threshold
        ]
        
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)            
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def test(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        
        

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='gini')

In [4]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()

In [5]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [6]:
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [7]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.37500500679
1.11724114418


## Проверка качества работы

In [8]:
gkf = KFold(n_splits=5, shuffle=True)

In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.929076245115
0.930905462709
0.934231312879
0.932027937141
0.934062279134


In [10]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.890371663757
0.894113245198
0.891743576952
0.891993015715
0.89793372968


## Проверка качества на произвольном числе кластеров

In [11]:
from sklearn.datasets import make_blobs
x, y = make_blobs(n_samples=300, n_features=3, centers=4, cluster_std=1,
center_box=(-10.0, 10.0), shuffle=False, random_state=1234)

In [12]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

1.0
1.0
0.983333333333
1.0
1.0


In [13]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.966666666667
0.983333333333
0.983333333333
1.0
1.0
